#### Import packages

In [255]:
import os
import re
import io

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import socket

from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer

from ipwhois import IPWhois

## Reads the dataset

#### Datasets directly from Netflow

In [256]:
filePath = "../../project_course_data/"
fileName = "owndata.txt"

with open(filePath + fileName, "r") as f:
    content = f.read().replace("->", " ").replace(" K ", "K ").replace(" M ", "M ").replace(" G ", "G ")
    
csvStringIO = io.StringIO(content)
columnNames = ["Datetime", "Time", "Duration",  "Proto", "Src IP Addr:Port", "Dst IP Addr:Port", "Packets", "Bytes", "Flows"]

# dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')

data = pd.read_csv(csvStringIO, sep = '\s+', names = columnNames, header = None, usecols=range(len(columnNames)), parse_dates=True, engine = "python")
data = data.iloc[1:-4] #Removes the summary lines and col names

data["Datetime"] += " " + data["Time"]
data = data.drop(columns = ["Time"])

data["Id"] = data.reset_index(drop = True).index


#### Function for debugging

In [257]:
def printRow(df, id):
    try:
        row = df[df["Id"] == id]
        for col, val in row.iloc[0].items():
            print(f"{col}: {val}")
    except KeyError:
        print(f"Row with id {id} not found in DataFrame.")

In [258]:
id = 332
printRow(data, id)

Datetime: 2023-11-22 13:41:55.315
Duration: 415.494
Proto: TCP
Src IP Addr:Port: 23.246.26.130:443
Dst IP Addr:Port: 192.168.8.235:64275
Packets: 700
Bytes: 1.1M
Flows: 1
Id: 332


## SORTING AND SPLITTING AND COMBINING

In [259]:
 # Convert the datetime column to pandas datetime format
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Sort the DataFrame by the datetime column
data = data.sort_values(by='Datetime')

# Split IP address and port to two columns, and drops the old column
data[['Src IP Addr', 'Src Port']] = data['Src IP Addr:Port'].str.split(':', n=1, expand=True)
data[['Dst IP Addr', 'Dst Port']] = data['Dst IP Addr:Port'].str.split(':', n=1, expand=True)
data = data.drop(columns=["Src IP Addr:Port", "Dst IP Addr:Port"])


In [260]:
printRow(data, id)

Datetime: 2023-11-22 13:41:55.315000
Duration: 415.494
Proto: TCP
Packets: 700
Bytes: 1.1M
Flows: 1
Id: 332
Src IP Addr: 23.246.26.130
Src Port: 443
Dst IP Addr: 192.168.8.235
Dst Port: 64275


#### Preprocess data

In [261]:
# Changes Duration column to float
data['Duration'] = data['Duration'].astype(float)
# Drop outliers
data = data.drop(data[data.Duration > 10000].index)

# Find clients IP address
client = data['Src IP Addr'].value_counts().idxmax().split('.')
client = '.'.join(client[:3])

# Initialize 'Host IP'-column from 'Src Ip Addr'
data['Host IP'] = data['Src IP Addr']
data['Client IP'] = data['Dst IP Addr']

# Removes all internal flows
# If the destination IP is not equal to the clients IP, adds it to 'Host IP'-column
for index, row in data.iterrows():
    if client in row['Dst IP Addr'] and client in row['Src IP Addr']:
        data.drop(index, inplace=True)
    elif client not in row['Dst IP Addr']:
        data.at[index, 'Host IP'] = row['Dst IP Addr']
        data.at[index, 'Client IP'] = row['Src IP Addr']

In [262]:
printRow(data, id)

Datetime: 2023-11-22 13:41:55.315000
Duration: 415.494
Proto: TCP
Packets: 700
Bytes: 1.1M
Flows: 1
Id: 332
Src IP Addr: 23.246.26.130
Src Port: 443
Dst IP Addr: 192.168.8.235
Dst Port: 64275
Host IP: 23.246.26.130
Client IP: 192.168.8.235


#### Prefix processing

In [263]:
def convert_bytes(value):
    value = str(value).upper()
    multipliers = {'K': 1024, 'M': 1024**2, 'G': 1024**3}

    if value[-1] in multipliers:
        return int(float(value[0:-1]) * multipliers[value[-1]])
    else:
        return int(value)

# Apply the conversion function to the 'Bytes' column
data['Bytes'] = data['Bytes'].apply(convert_bytes)

In [264]:
printRow(data, id)

Datetime: 2023-11-22 13:41:55.315000
Duration: 415.494
Proto: TCP
Packets: 700
Bytes: 1153433
Flows: 1
Id: 332
Src IP Addr: 23.246.26.130
Src Port: 443
Dst IP Addr: 192.168.8.235
Dst Port: 64275
Host IP: 23.246.26.130
Client IP: 192.168.8.235


#### Reverse DNS-lookup

In [265]:
# Find all unique addresses
unique_ip = data['Host IP'].unique()

# Creates a new dataframe
data_DNS = pd.DataFrame(columns=['IP', 'Host'])

data_DNS['IP']=unique_ip
host = []

# for-loop for doing reverse DNS lookup
i=0
for ip in unique_ip:
    try:
        host_name = socket.gethostbyaddr(ip)[0]
        host.append(host_name)
    except socket.herror:
        host.append(None)
    
    if i % 100 == 0: # Used to keep track how far along we've come
        print(f"{i} / {len(unique_ip)}")
    i += 1

# Adds the corresponding domain names to the IP-addresses and creates a CSV-file
data_DNS['Host'] = host
# data_DNS.to_csv('./host_names', index=False)

0 / 225
100 / 225
200 / 225


In [266]:
DNS_dict = {}
for index, row in data_DNS.iterrows():
    DNS_dict[row["IP"]] = row["Host"]

print(DNS_dict)


dataWithDomains = data.copy()
dataWithDomains["Domain Name"] = dataWithDomains.apply(lambda row: DNS_dict[row["Host IP"]], axis= 1)


{'3.67.35.217': 'ec2-3-67-35-217.eu-central-1.compute.amazonaws.com', '13.107.42.18': None, '3.68.18.70': 'ec2-3-68-18-70.eu-central-1.compute.amazonaws.com', '20.54.37.64': None, '95.101.133.144': 'a95-101-133-144.deploy.static.akamaitechnologies.com', '13.107.21.200': None, '13.107.246.53': None, '13.107.5.93': None, '150.171.22.254': None, '216.58.207.202': 'arn11s04-in-f10.1e100.net', '142.250.74.138': 'arn11s11-in-f10.1e100.net', '104.18.32.7': None, '172.217.21.170': 'fra07s64-in-f170.1e100.net', '151.101.84.193': None, '142.250.74.129': 'arn11s11-in-f1.1e100.net', '142.250.74.174': 'arn11s12-in-f14.1e100.net', '20.42.65.91': None, '142.250.74.46': 'arn09s22-in-f14.1e100.net', '108.177.14.188': 'lt-in-f188.1e100.net', '95.101.133.43': 'a95-101-133-43.deploy.static.akamaitechnologies.com', '162.159.134.234': None, '20.54.37.73': None, '2.22.42.56': 'a2-22-42-56.deploy.static.akamaitechnologies.com', '162.159.136.232': None, '68.232.34.200': None, '13.69.239.73': None, '216.239.32.

#### Changes Protocol into feature-columns

In [268]:
dataWithDummies = dataWithDomains.copy()

# Get the dummies and store it in a variable
dummies = pd.get_dummies(dataWithDummies.Proto).astype(int)
 
# Concatenate the dummies to original dataframe
dataWithDummies = pd.concat([dataWithDummies, dummies], axis='columns')

# drop the values
dataWithDummies = dataWithDummies.drop(['Proto'], axis='columns')
dataWithDummies = dataWithDummies.fillna('0')

                   Datetime  Duration Packets  Bytes Flows   Id  \
4   2023-11-22 12:17:01.655   135.108     100   4000     1    3   
5   2023-11-22 12:17:01.655   135.108     100   4600     1    4   
2   2023-11-22 12:18:39.247     0.000     100  97000     1    1   
3   2023-11-22 12:18:41.283     0.000     100   9400     1    2   
6   2023-11-22 12:19:58.622     0.000     100  14100     1    5   
..                      ...       ...     ...    ...   ...  ...   
694 2023-11-22 16:18:03.022     0.000     100   5200     1  693   
695 2023-11-22 16:18:06.920     0.000     100   6000     1  694   
699 2023-11-22 16:19:33.194     0.000     100  10500     1  698   
701 2023-11-22 16:23:02.311     0.000     100  20300     1  700   
704 2023-11-22 16:24:46.660     0.000     100  50900     1  703   

       Src IP Addr Src Port      Dst IP Addr Dst Port          Host IP  \
4      3.67.35.217      443    192.168.8.235    63931      3.67.35.217   
5    192.168.8.235    63931      3.67.35.217   

### Change PORT into feature columns

In [269]:
#  Creates a column for non-client ports
dataWithDummies['Host Port'] = None

# Adds the non-client port to the new column by checking that the ports (Src & Dst) does not contain the client IP
for index, row in dataWithDummies.iterrows():
    if row["Host IP"] ==  row['Src IP Addr']:
        dataWithDummies.at[index, 'Host Port'] = row['Src Port']
    else:
        dataWithDummies.at[index, 'Host Port'] = row['Dst Port'] 

# Get the dummies and store it in a variable
dummies = pd.get_dummies(dataWithDummies["Host Port"]).astype(int)
 
# Concatenate the dummies to original dataframe
dataWithDummies = pd.concat([dataWithDummies, dummies], axis='columns')

# drop the values
dataWithDummies = dataWithDummies.fillna('0')
        

                   Datetime  Duration Packets  Bytes Flows   Id  \
4   2023-11-22 12:17:01.655   135.108     100   4000     1    3   
5   2023-11-22 12:17:01.655   135.108     100   4600     1    4   
2   2023-11-22 12:18:39.247     0.000     100  97000     1    1   
3   2023-11-22 12:18:41.283     0.000     100   9400     1    2   
6   2023-11-22 12:19:58.622     0.000     100  14100     1    5   
..                      ...       ...     ...    ...   ...  ...   
694 2023-11-22 16:18:03.022     0.000     100   5200     1  693   
695 2023-11-22 16:18:06.920     0.000     100   6000     1  694   
699 2023-11-22 16:19:33.194     0.000     100  10500     1  698   
701 2023-11-22 16:23:02.311     0.000     100  20300     1  700   
704 2023-11-22 16:24:46.660     0.000     100  50900     1  703   

       Src IP Addr Src Port      Dst IP Addr Dst Port  ... 27036 27043 27047  \
4      3.67.35.217      443    192.168.8.235    63931  ...     0     0     0   
5    192.168.8.235    63931      3.

In [270]:
print("A row of data after port dummies:\n")
printRow(dataWithDummies, id)

A row of data after port dummies:

Datetime: 2023-11-22 13:41:55.315000
Duration: 415.494
Packets: 700
Bytes: 1153433
Flows: 1
Id: 332
Src IP Addr: 23.246.26.130
Src Port: 443
Dst IP Addr: 192.168.8.235
Dst Port: 64275
Host IP: 23.246.26.130
Client IP: 192.168.8.235
Domain Name: ipv4-c091-arn001-ix.1.oca.nflxvideo.net
TCP: 1
UDP: 0
Host Port: 443
1900: 0
27018: 0
27025: 0
27036: 0
27043: 0
27047: 0
27051: 0
27053: 0
27057: 0
27060: 0
443: 1
5228: 0
80: 0


### BAG OF WORDS FOR THE DOMAIN NAMES

In [271]:
dataWithBOW = dataWithDummies.copy()

# Create and fit vectorizer
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(dataWithBOW['Domain Name'])

# vectorize
df_bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# find columns with only numeric names and drop them
numeric_columns = df_bow.columns[df_bow.columns.str.isnumeric()]
df_bow.drop(numeric_columns, axis=1, inplace=True)

dataWithBOW.reset_index(drop=True, inplace=True)
df_bow.reset_index(drop=True, inplace=True)
dataWithBOW = pd.concat([dataWithBOW, df_bow], axis=1)

In [272]:
print("A row of data after Bag of Words:\n")
printRow(dataWithBOW, id)

A row of data after Bag of Words:

Datetime: 2023-11-22 13:41:55.315000
Duration: 415.494
Packets: 700
Bytes: 1153433
Flows: 1
Id: 332
Src IP Addr: 23.246.26.130
Src Port: 443
Dst IP Addr: 192.168.8.235
Dst Port: 64275
Host IP: 23.246.26.130
Client IP: 192.168.8.235
Domain Name: ipv4-c091-arn001-ix.1.oca.nflxvideo.net
TCP: 1
UDP: 0
Host Port: 443
1900: 0
27018: 0
27025: 0
27036: 0
27043: 0
27047: 0
27051: 0
27053: 0
27057: 0
27060: 0
443: 1
5228: 0
80: 0
1drv: 0
1e100: 0
833aec: 0
a104: 0
a184: 0
a2: 0
a23: 0
a95: 0
akamaitechnologies: 0
amazonaws: 0
ams15s51: 0
arn001: 1
arn04: 0
arn09s18: 0
arn09s19: 0
arn09s20: 0
arn09s21: 0
arn09s22: 0
arn09s23: 0
arn09s25: 0
arn09s26: 0
arn09s27: 0
arn1: 0
arn11s03: 0
arn11s04: 0
arn11s09: 0
arn11s10: 0
arn11s11: 0
arn11s12: 0
arn11s13: 0
arn11s14: 0
arn2: 0
arn54: 0
arn56: 0
bc: 0
berlin: 0
bkk03s02: 0
c061: 0
c062: 0
c063: 0
c066: 0
c069: 0
c073: 0
c081: 0
c087: 0
c090: 0
c091: 1
c093: 0
c095: 0
c097: 0
c098: 0
c099: 0
c100: 0
c101: 0
c102: 0
c1

### CERTIFICATE LOOKUP

In [273]:
# WHOIS FETCH FUNCTION
def get_ip_info(ip):
    
    ipwhois_obj = IPWhois(ip)

    result = ipwhois_obj.lookup_rdap()

    # Available information from ipwhois:

    # print("IP Address:", result['query'])
    # print("ASN:", result['asn'])
    # print("CIDR:", result['asn_cidr'])
    # print("Name:", result['network']['name'])
    # print("Country:", result['asn_country_code'])
    # print("Description:", result['asn_description'])

    return {"name": result['network']['name'], "country": result['asn_country_code']}

In [274]:
dataWithWhoIs = dataWithBOW.copy()

sz = len(dataWithWhoIs)
t = 0
memo = {}
## ITERATING THROUGH DATASET / CERTIFICATE LOOKUP OF IP
for index, row in dataWithWhoIs.iterrows():

    host_IP = row["Host IP"]

    if(host_IP not in memo): # NEW IP
        print("not in")
        try: 
            # fetch ipwhois info
            whoIsResult = get_ip_info(row["Host IP"])

            # add info
            dataWithWhoIs.at[index, "ipwhois_name"] = whoIsResult["name"]
            dataWithWhoIs.at[index, "ipwhois_country"] = whoIsResult["country"]

            # save for later
            memo[host_IP] = whoIsResult

        except:
            # default option
            dataWithWhoIs.at[index, "ipwhois_name"] = np.nan
            dataWithWhoIs.at[index, "ipwhois_country"] = np.nan

    else: # IP ALREADY CHECKED
        # get saved whois info
        whoIsResult = memo[host_IP]

        # add info
        dataWithWhoIs.at[index, "ipwhois_name"] = whoIsResult["name"]
        dataWithWhoIs.at[index, "ipwhois_country"] = whoIsResult["country"]

    t+=1
    
    print(f"{t} / {sz}")

not in
1 / 683
2 / 683
not in
3 / 683
not in
4 / 683
not in
5 / 683
not in
6 / 683
7 / 683
not in
8 / 683
not in
9 / 683
not in
10 / 683
11 / 683
12 / 683
not in
13 / 683
14 / 683
15 / 683
not in
16 / 683
not in
17 / 683
not in
18 / 683
not in
19 / 683
not in
20 / 683
not in
21 / 683
22 / 683
not in
23 / 683
24 / 683
25 / 683
26 / 683
not in
27 / 683
not in
28 / 683
not in
29 / 683
not in
30 / 683
not in
31 / 683
32 / 683
not in
33 / 683
not in
34 / 683
35 / 683
not in
36 / 683
not in
37 / 683
38 / 683
39 / 683
40 / 683
not in
41 / 683
42 / 683
43 / 683
not in
44 / 683
not in
45 / 683
not in
46 / 683
47 / 683
48 / 683
not in
49 / 683
50 / 683
not in
51 / 683
not in
52 / 683
not in
53 / 683
not in
54 / 683
not in
55 / 683
not in
56 / 683
57 / 683
not in
58 / 683
59 / 683
not in
60 / 683
not in
61 / 683
62 / 683
not in
63 / 683
not in
64 / 683
65 / 683
not in
66 / 683
not in
67 / 683
not in
68 / 683
69 / 683
70 / 683
not in
71 / 683
72 / 683
73 / 683
not in
74 / 683
not in
75 / 683
76 / 

In [275]:
# MAKE DUMMIES FROM WHOIS DATA

# get the dummies and store it in a variable
dummies_name = pd.get_dummies(dataWithWhoIs.ipwhois_name).astype(int)
dummies_country = pd.get_dummies(dataWithWhoIs.ipwhois_country).astype(int)

 
# Concatenate the dummies to original dataframe
dataWithWhoIs = pd.concat([dataWithWhoIs, dummies_name], axis='columns')
dataWithWhoIs = pd.concat([dataWithWhoIs, dummies_country], axis='columns')


# drop the values
dataWithWhoIs = dataWithWhoIs.drop(["ipwhois_name", "ipwhois_country"], axis='columns')
dataWithWhoIs = dataWithWhoIs.fillna(0)

In [276]:
print("A row of data after whois:\n")
printRow(dataWithWhoIs, id)

A row of data after whois:

Datetime: 2023-11-22 13:41:55.315000
Duration: 415.494
Packets: 700
Bytes: 1153433
Flows: 1
Id: 332
Src IP Addr: 23.246.26.130
Src Port: 443
Dst IP Addr: 192.168.8.235
Dst Port: 64275
Host IP: 23.246.26.130
Client IP: 192.168.8.235
Domain Name: ipv4-c091-arn001-ix.1.oca.nflxvideo.net
TCP: 1
UDP: 0
Host Port: 443
1900: 0
27018: 0
27025: 0
27036: 0
27043: 0
27047: 0
27051: 0
27053: 0
27057: 0
27060: 0
443: 1
5228: 0
80: 0
1drv: 0
1e100: 0
833aec: 0
a104: 0
a184: 0
a2: 0
a23: 0
a95: 0
akamaitechnologies: 0
amazonaws: 0
ams15s51: 0
arn001: 1
arn04: 0
arn09s18: 0
arn09s19: 0
arn09s20: 0
arn09s21: 0
arn09s22: 0
arn09s23: 0
arn09s25: 0
arn09s26: 0
arn09s27: 0
arn1: 0
arn11s03: 0
arn11s04: 0
arn11s09: 0
arn11s10: 0
arn11s11: 0
arn11s12: 0
arn11s13: 0
arn11s14: 0
arn2: 0
arn54: 0
arn56: 0
bc: 0
berlin: 0
bkk03s02: 0
c061: 0
c062: 0
c063: 0
c066: 0
c069: 0
c073: 0
c081: 0
c087: 0
c090: 0
c091: 1
c093: 0
c095: 0
c097: 0
c098: 0
c099: 0
c100: 0
c101: 0
c102: 0
c104: 0
c

### Min Max normalization

In [286]:
dataNorm = dataWithWhoIs.copy()
dataNorm["Packets"] = pd.to_numeric(dataNorm["Packets"])

columns = ['Bytes', 'Duration', 'Packets']
for column in columns:
    dataNorm[column] = (dataNorm[column] - dataNorm[column].min()) / (dataNorm[column].max() - dataNorm[column].min()) 

In [287]:
print("A row of data after normalization:\n")
printRow(dataNorm, id)

A row of data after normalization:

Datetime: 2023-11-22 13:41:55.315000
Duration: 0.15882470315979716
Packets: 0.10714285714285714
Bytes: 0.15232858986658313
Flows: 1
Id: 332
Src IP Addr: 23.246.26.130
Src Port: 443
Dst IP Addr: 192.168.8.235
Dst Port: 64275
Host IP: 23.246.26.130
Client IP: 192.168.8.235
Domain Name: ipv4-c091-arn001-ix.1.oca.nflxvideo.net
TCP: 1
UDP: 0
Host Port: 443
1900: 0
27018: 0
27025: 0
27036: 0
27043: 0
27047: 0
27051: 0
27053: 0
27057: 0
27060: 0
443: 1
5228: 0
80: 0
1drv: 0
1e100: 0
833aec: 0
a104: 0
a184: 0
a2: 0
a23: 0
a95: 0
akamaitechnologies: 0
amazonaws: 0
ams15s51: 0
arn001: 1
arn04: 0
arn09s18: 0
arn09s19: 0
arn09s20: 0
arn09s21: 0
arn09s22: 0
arn09s23: 0
arn09s25: 0
arn09s26: 0
arn09s27: 0
arn1: 0
arn11s03: 0
arn11s04: 0
arn11s09: 0
arn11s10: 0
arn11s11: 0
arn11s12: 0
arn11s13: 0
arn11s14: 0
arn2: 0
arn54: 0
arn56: 0
bc: 0
berlin: 0
bkk03s02: 0
c061: 0
c062: 0
c063: 0
c066: 0
c069: 0
c073: 0
c081: 0
c087: 0
c090: 0
c091: 1
c093: 0
c095: 0
c097: 0
c

### SAVE TO FILE

In [288]:
newFileName = "preprocessedData.csv"
dataWithWhoIs.to_csv(filePath + newFileName, sep="\t", index = False)